In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sys, os
import shutil
import urllib
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.datastore import Datastore
import platform,  dotenv

In [3]:
print("Azure ML SDK Version: ", azureml.core.VERSION)
platform.platform()
os.getcwd()

Azure ML SDK Version:  1.0.74


'Linux-4.15.0-1063-azure-x86_64-with-debian-10.1'

'/workspace/code'

In [4]:
nihchestxraymetadata_ds_name='nihchestxraymetadata_ds'
nihchestxrayimages_ds_name='nihchestxrayimages_ds'
experiment_name = "chestxray-keras"

## Credentials file

In [5]:
dotenv_file_path = './../not_shared/' 
os.makedirs(dotenv_file_path, exist_ok=True)
dotenv_file_path = os.path.join(dotenv_file_path, 'general.env')
dotenv_file_path
! sudo touch $dotenv_file_path

'./../not_shared/general.env'

In [6]:
subscription_id=""
resource_group=""
rsg_region="eastus"

workspace_name = 'ghiordanazurechstxrayws'
workspace_region = rsg_region

sa_name=""
sa_key=""
sa_container_name="nihchestxraydata"
sa_container_name_compressed_data="nihchestxraydatacompressed"
sa_container_name_images="nihchestxrayimages"

gpu_cluster_name = 'azuremlchestxray'
gpucluster_admin_user_name=""
gpucluster_admin_user_password = ""

docker_login="georgedockeraccount"
docker_password=""

In [7]:
# response = dotenv.set_key(dotenv_file_path, 'subscription_id', subscription_id)
# response = dotenv.set_key(dotenv_file_path, 'resource_group', resource_group)
# response = dotenv.set_key(dotenv_file_path, 'rsg_region', rsg_region)

# response = dotenv.set_key(dotenv_file_path, 'workspace_name', workspace_name)
# response = dotenv.set_key(dotenv_file_path, 'workspace_region', workspace_region)

# response = dotenv.set_key(dotenv_file_path, 'sa_name', sa_name)
# response = dotenv.set_key(dotenv_file_path, 'sa_key', sa_key)
# response = dotenv.set_key(dotenv_file_path, 'sa_container_name', sa_container_name)
# response = dotenv.set_key(dotenv_file_path, 'sa_container_name_compressed_data', sa_container_name_compressed_data)
# response = dotenv.set_key(dotenv_file_path, 'sa_container_name_images', sa_container_name_images)

# response = dotenv.set_key(dotenv_file_path, 'gpu_cluster_name', gpu_cluster_name)
# response = dotenv.set_key(dotenv_file_path, 'gpucluster_admin_user_name', gpucluster_admin_user_name)
# response = dotenv.set_key(dotenv_file_path, 'gpucluster_admin_user_password', gpucluster_admin_user_password)

# response = dotenv.set_key(dotenv_file_path, 'docker_login', docker_login)
# response = dotenv.set_key(dotenv_file_path, 'docker_password', docker_password)

In [8]:
%load_ext dotenv
%dotenv $dotenv_file_path -o

In [9]:
# !cat $dotenv_file_path

## Set Up Workspace

In [10]:
subscription_id = os.getenv('subscription_id')
# # print a bit of subscription ID, to show dotenv file was found and loaded 
# subscription_id[:2]

crt_resource_group  = os.getenv('resource_group')
crt_workspace_name = os.getenv('workspace_name')
crt_workspace_region = os.getenv('workspace_region')

In [11]:
try:
    ws1 = Workspace(
        subscription_id = subscription_id, 
        resource_group = crt_resource_group, 
        workspace_name = crt_workspace_name)
    print("Workspace configuration loading succeeded. ")
#     ws1.write_config()
    del ws1 # ws will be (re)created later using from_config() function
except Exception as e :
    print('Exception msg: {}'.format(str(e )))
    print("Workspace not accessible. Will create a new workspace below")
    
    workspace_region = crt_workspace_region

    # Create the workspace using the specified parameters
    ws2 = Workspace.create(name = crt_workspace_name,
                          subscription_id = subscription_id,
                          resource_group = crt_resource_group, 
                          location = workspace_region,
                          create_resource_group = False,
                          exist_ok = False)
    ws2.get_details()

    # persist the subscription id, resource group name, and workspace name in aml_config/config.json.
    ws2.write_config()
    
    #Delete ws2 and use ws = Workspace.from_config() as shwon below to recover the ws, rather than rely on what we get from one time creation
    del ws2

Workspace configuration loading succeeded. 


In [12]:
#later accesses
ws = Workspace.from_config()

### Make new Experiment in workspace.

In [13]:
exp = Experiment(workspace=ws, name=experiment_name)

## Set up a new datastore

Create your own access keys file to store your secret credentials.

In [14]:
ds1 = Datastore.register_azure_blob_container(workspace=ws, 
                                             datastore_name=nihchestxraymetadata_ds_name, 
                                             container_name=os.getenv('sa_container_name'),
                                             account_name=os.getenv('sa_name'), 
                                             account_key=os.getenv('sa_key'),
                                             create_if_not_exists=False,
                                             overwrite=True)
ds2 = Datastore.register_azure_blob_container(workspace=ws, 
                                             datastore_name=nihchestxrayimages_ds_name, 
                                             container_name=os.getenv('sa_container_name_images'),
                                             account_name=os.getenv('sa_name'), 
                                             account_key=os.getenv('sa_key'),
                                             create_if_not_exists=False,
                                             overwrite=True)



In [15]:
#list all datastores registered in current workspace
datastores = ws.datastores
for name, ds in datastores.items():
    print(name, ds.datastore_type)
    
Datastore.get(ws, nihchestxraymetadata_ds_name).as_mount()
Datastore.get(ws, nihchestxrayimages_ds_name).as_mount()

workspaceblobstore AzureBlob
workspacefilestore AzureFile
nihchestxraymetadata_ds AzureBlob
nihchestxrayimages_ds AzureBlob


$AZUREML_DATAREFERENCE_nihchestxraymetadata_ds

$AZUREML_DATAREFERENCE_nihchestxrayimages_ds

On the first run-through, upload images to datastore.

In [16]:
# ds.upload(src_dir = './../../../data', target_path = 'data', show_progress = True)

## Set Compute Target

In [19]:
# Choose a name for your GPU cluster
gpu_cluster_name = "gpucluster"

# Verify that cluster does not exist already
max_nodes_value = 2
min_nodes_value = 0
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print("Found existing gpu cluster")
except ComputeTargetException:
    print("Could not find ComputeTarget cluster!")
    
#     Create a new gpucluster using code below
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC24s_v2', # "Standard_NC24s_v3",
                                                           min_nodes=0,
                                                           max_nodes=max_nodes_value)
    # Create the cluster with the specified name and configuration
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

    # Wait for the cluster to complete, show the output log
    gpu_cluster.wait_for_completion(show_output=True)

# for demo purposes, this is how you resize the cluster    
gpu_cluster.update(min_nodes=min_nodes_value, max_nodes=max_nodes_value, idle_seconds_before_scaledown=1200)


Found existing gpu cluster


## Set up script directory

In [ ]:
script_folder = './xray_scripts'

In [ ]:
get_data_filename = 'get_data.py'
train_script_filename = 'train.py'

## Create script to copy and process data from Azure Blob Storage

Pre-processing code organizes the images and labels, removes bad images, and splits the data into training, validation, and testing sets.

In [ ]:
%%writefile {os.path.join(script_folder, get_data_filename)}

import sys
import os
import subprocess
import src.azure_chestxray_utils

import pickle
import random
import re
import tqdm

import cv2
import numpy as np
import pandas as pd
import sklearn.model_selection 


def preprocess_data(base_dir):
    #TODO -- get constants from separate file
    prj_consts = src.azure_chestxray_utils.chestxray_consts()

    data_base_input_dir=os.path.join(base_dir, os.path.join(*(prj_consts.BASE_INPUT_DIR_list)))
    data_base_output_dir=os.path.join(base_dir, os.path.join(*(prj_consts.BASE_OUTPUT_DIR_list))) 
    nih_chest_xray_data_dir=os.path.join(data_base_input_dir, os.path.join(*(prj_consts.ChestXray_IMAGES_DIR_list)))
    
    data_partitions_dir=os.path.join(data_base_output_dir, os.path.join(*(prj_consts.DATA_PARTITIONS_DIR_list))) 

    partition_path = os.path.join(data_partitions_dir, 'partition14_unormalized_cleaned.pickle')
    label_path = os.path.join(data_partitions_dir,'labels14_unormalized_cleaned.pickle')

    # global variables

    weights_dir = os.path.join(data_base_output_dir, os.path.join(*(prj_consts.MODEL_WEIGHTS_DIR_list))) 
    
    #check number of images
#     orig_images_no = subprocess.call("find $nih_chest_xray_data_dir -type f | wc -l")
#     print("orig images number:{} ".format(orig_images_no))
    
    #Train/Validation/Test Data partitioning
    total_patient_number = 30805
    NIH_annotated_file = 'BBox_List_2017.csv' # exclude from train pathology annotated by radiologists 
    manually_selected_bad_images_file = 'blacklist.csv'# exclude what viusally looks like bad images
    
    patient_id_original = [i for i in range(1,total_patient_number + 1)]
    
    other_data_dir=os.path.join(nih_chest_xray_data_dir, os.path.join(*(['..','..'])))
    
    # ignored images list is used later, since this is not a patient ID level issue
    ignored_images_set = set()
    with open(os.path.join(other_data_dir, manually_selected_bad_images_file), 'r') as f:
        for line in f:
            # delete the last char which is \n
            ignored_images_set.add(line[:-1])
            if int(line[:-9]) >= 30805:
                print(line[:-1])

    bbox_df = pd.read_csv(os.path.join(other_data_dir, NIH_annotated_file))
    bbox_patient_index_df = bbox_df['Image Index'].str.slice(3, 8)

    bbox_patient_index_list = []
    for index, item in bbox_patient_index_df.iteritems():
        bbox_patient_index_list.append(int(item))

    patient_id = list(set(patient_id_original) - set(bbox_patient_index_list))
    print("len of original patient id is", len(patient_id_original))
    print("len of cleaned patient id is", len(patient_id))
    print("len of unique patient id with annotated data", 
          len(list(set(bbox_patient_index_list))))
    print("len of patient id with annotated data",bbox_df.shape[0])
    
    random.seed(0)
    random.shuffle(patient_id)

    print("first ten patient ids are", patient_id[:10])

    # training:valid:test=7:1:2
    patient_id_train = patient_id[:int(total_patient_number * 0.7)]
    patient_id_valid = patient_id[int(total_patient_number * 0.7):int(total_patient_number * 0.8)]
    # get the rest of the patient_id as the test set
    patient_id_test = patient_id[int(total_patient_number * 0.8):]
    patient_id_test.extend(bbox_patient_index_list)
    patient_id_test = list(set(patient_id_test))


    print("train:{} valid:{} test:{}".format(len(patient_id_train), len(patient_id_valid), len(patient_id_test)))

    # test_set = test_set+left_out_patient_id
    # print("train:{} valid:{} test:{}".format(len(train_set), len(valid_set), len(test_set)))
    
    pathologies_name_list = prj_consts.DISEASE_list
    NIH_patients_and_labels_file = 'Data_Entry_2017.csv'
    
    labels_df = pd.read_csv(os.path.join(other_data_dir, NIH_patients_and_labels_file))
    
    # # create and save train/test/validation partitions list
    
    def process_images(current_df, patient_ids):
        image_name_index = []
        image_labels = {}
        for individual_patient in tqdm.tqdm(patient_ids):
            for _, row in current_df[current_df['Patient ID'] == individual_patient].iterrows():
                processed_image_name = row['Image Index']
                if processed_image_name in ignored_images_set:
                    pass
                else:
                    image_name_index.append(processed_image_name)
                    image_labels[processed_image_name] = np.zeros(14, dtype=np.uint8)
                    for disease_index, ele in enumerate(pathologies_name_list):
                        if re.search(ele, row['Finding Labels'], re.IGNORECASE):
                            image_labels[processed_image_name][disease_index] = 1
                        else:
                            # redundant code but just to make it more readable
                            image_labels[processed_image_name][disease_index] = 0
                    # print("processed", row['Image Index'])
        return image_name_index, image_labels

    train_data_index, train_labels = process_images(labels_df, patient_id_train)
    valid_data_index, valid_labels = process_images(labels_df, patient_id_valid)
    test_data_index, test_labels = process_images(labels_df, patient_id_test)

    print("train, valid, test image number is:", len(train_data_index), len(valid_data_index), len(test_data_index))

    # save the data
    labels_all = {}
    labels_all.update(train_labels)
    labels_all.update(valid_labels)
    labels_all.update(test_labels)

    partition_dict = {'train': train_data_index, 'test': test_data_index, 'valid': valid_data_index}

    with open(os.path.join(data_partitions_dir,'labels14_unormalized_cleaned.pickle'), 'wb') as f:
        pickle.dump(labels_all, f)

    with open(os.path.join(data_partitions_dir,'partition14_unormalized_cleaned.pickle'), 'wb') as f:
        pickle.dump(partition_dict, f)

    # also save the patient id partitions for pytorch training    
    with open(os.path.join(data_partitions_dir,'train_test_valid_data_partitions.pickle'), 'wb') as f:
        pickle.dump([patient_id_train,patient_id_valid,
                     patient_id_test,
                    list(set(bbox_patient_index_list))], f)    

    

## Create a training script

Save the training script to a file to run on the compute source later.

In [ ]:
%%writefile {os.path.join(script_folder, train_script_filename)}

import sys, os

parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder')
args = parser.parse_args()
print(args)
print('Data folder is at:', args.data_folder)
base_dir = args.data_folder
print('List all files: ', os.listdir(args.data_folder))

import src.azure_chestxray_utils as azure_chestxray_utils
import src.azure_chestxray_keras_utils as azure_chestxray_keras_utils

import subprocess

import imgaug as ia
from imgaug import augmenters as iaa
ia.seed(1)

import cv2
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, Callback, ModelCheckpoint
import numpy as np
import pickle
from keras_contrib.applications.densenet import DenseNetImageNet121
from keras.layers import Dense
from keras.models import Model
from keras.utils import multi_gpu_model
from tensorflow.python.client import device_lib
import warnings
from keras.utils import Sequence
import tensorflow as tf
import get_data


def get_available_gpus():
    """

    Returns: list of GPUs available in the system

    """
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
#     try:
#         out_str = subprocess.run(["nvidia-smi", "--query-gpu=gpu_name", "--format=csv"], stdout=subprocess.PIPE).stdout
#         out_list = out_str.decode("utf-8").split('\n')
#         out_list = out_list[1:-1]
#         return out_list
#     except Exception as e:
#         print(e)


# multi GPU model checkpoint. copied from https://github.com/keras-team/keras/issues/8463
class MultiGPUCheckpointCallback(Callback):

    def __init__(self, filepath, base_model, monitor='val_loss', verbose=0,
                 save_best_only=False, save_weights_only=False,
                 mode='auto', period=1):
        super(MultiGPUCheckpointCallback, self).__init__()
        self.base_model = base_model
        self.monitor = monitor
        self.verbose = verbose
        self.filepath = filepath
        self.save_best_only = save_best_only
        self.save_weights_only = save_weights_only
        self.period = period
        self.epochs_since_last_save = 0

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('ModelCheckpoint mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor or self.monitor.startswith('fmeasure'):
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0
            filepath = self.filepath.format(epoch=epoch + 1, **logs)
            if self.save_best_only:
                current = logs.get(self.monitor)
                if current is None:
                    warnings.warn('Can save best model only with %s available, '
                                  'skipping.' % (self.monitor), RuntimeWarning)
                else:
                    if self.monitor_op(current, self.best):
                        if self.verbose > 0:
                            print('Epoch %05d: %s improved from %0.5f to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch + 1, self.monitor, self.best,
                                     current, filepath))
                        self.best = current
                        if self.save_weights_only:
                            self.base_model.save_weights(filepath, overwrite=True)
                        else:
                            self.base_model.save(filepath, overwrite=True)
                    else:
                        if self.verbose > 0:
                            print('Epoch %05d: %s did not improve' %
                                  (epoch + 1, self.monitor))
            else:
                if self.verbose > 0:
                    print('Epoch %05d: saving model to %s' % (epoch + 1, filepath))
                if self.save_weights_only:
                    self.base_model.save_weights(filepath, overwrite=True)
                else:
                    self.base_model.save(filepath, overwrite=True)

# generator for train and validation data
# use the Sequence class per issue https://github.com/keras-team/keras/issues/1638
class DataGenSequence(Sequence):
    def __init__(self, labels, image_file_index, current_state, batch_size):
        self.batch_size = batch_size
        print("BATCH SIZE 2", self.batch_size)
        self.labels = labels
        self.img_file_index = image_file_index
        self.current_state = current_state
        self.len = len(self.img_file_index) // self.batch_size
        print("for DataGenSequence", current_state, "total rows are:", len(self.img_file_index), ", len is", self.len)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        # print("loading data segmentation", idx)
        # make sure each batch size has the same amount of data
        current_batch = self.img_file_index[idx * self.batch_size: (idx + 1) * self.batch_size]
        X = np.empty((self.batch_size, resized_height, resized_width, num_channel))
        y = np.empty((self.batch_size, num_classes))

        for i, image_name in enumerate(current_batch):
            path = os.path.join(nih_chest_xray_data_dir, image_name)
            # loading data

            img = cv2.resize(cv2.imread(path), (resized_height, resized_width)).astype(np.float32)
            X[i, :, :, :] = img
            y[i, :] = labels[image_name]

            # only do random flipping in training status
        if self.current_state == 'train':
            x_augmented = seq.augment_images(X)
        else:
            x_augmented = X

        return x_augmented, y


# loss function
def unweighted_binary_crossentropy(y_true, y_pred):
    """
    Args:
        y_true: true labels
        y_pred: predicted labels

    Returns: the sum of binary cross entropy loss across all the classes

    """
    return K.sum(K.binary_crossentropy(y_true, y_pred))


def build_model():
    """

    Returns: a model with specified weights

    """
    # define the model, use pre-trained weights for image_net
    base_model = DenseNetImageNet121(input_shape=(224, 224, 3),
                                     weights='imagenet',
                                     include_top=False,
                                     pooling='avg')

    x = base_model.output
    predictions = Dense(14, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


if __name__ == "__main__":
    


    #call get_data script
#     base_dir = get_data.download_data()
    print("BASE DIR:", base_dir)
    get_data.preprocess_data(base_dir)
    
    # create the file path variables 
    # paths are typically container level dirs mapped to a host dir for data persistence.
    prj_consts = azure_chestxray_utils.chestxray_consts()

    data_base_input_dir=os.path.join(base_dir, os.path.join(*(prj_consts.BASE_INPUT_DIR_list)))
    data_base_output_dir=os.path.join(base_dir, os.path.join(*(prj_consts.BASE_OUTPUT_DIR_list))) 

    # data used for training
    nih_chest_xray_data_dir=os.path.join(data_base_input_dir, 
                                        os.path.join(*(prj_consts.ChestXray_IMAGES_DIR_list)))

    data_partitions_dir=os.path.join(data_base_output_dir, os.path.join(*(prj_consts.DATA_PARTITIONS_DIR_list))) 

    partition_path = os.path.join(data_partitions_dir, 'partition14_unormalized_cleaned.pickle')
    label_path = os.path.join(data_partitions_dir,'labels14_unormalized_cleaned.pickle')

    # global variables

    weights_dir = os.path.join(data_base_output_dir, os.path.join(*(prj_consts.MODEL_WEIGHTS_DIR_list))) 
    os.makedirs(weights_dir, exist_ok = True)

    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
    # get number of available GPUs
    num_gpu = len(get_available_gpus())
#     num_gpu = 1
    # keras multi_gpu_model slices the data to different GPUs. see https://keras.io/utils/#multi_gpu_model for more details.
    batch_size =num_gpu *  48 # 64 seems t obe too much on NC12 
    print("NUM GPU:", num_gpu)

    # make force_restart = False if you continue a previous train session, make it True to start from scratch
    force_restart = False

    initial_lr = 0.001
    resized_height = 224
    resized_width = 224
    # resized_height = prj_consts.CHESTXRAY_MODEL_EXPECTED_IMAGE_HEIGHT
    # resized_width = prj_consts.CHESTXRAY_MODEL_EXPECTED_IMAGE_WIDTH
    num_channel = 3
    num_classes = 14
    epochs = 250

    seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontal flips
    iaa.Affine(rotate=(-15, 15)),  # random rotate image
    iaa.Affine(scale=(0.8, 1.1)),  # randomly scale the image
    ], random_order=True)  # apply augmenters in random order

    if num_gpu > 1:
        print("using", num_gpu, "GPUs")
        # build model
        with tf.device('/cpu:0'):
            model_single_gpu = build_model()
        # model_single_gpu.load_weights(weights_path)

        # convert to multi-gpu model
        model_multi_gpu = multi_gpu_model(model_single_gpu, gpus=num_gpu)
        model_checkpoint = MultiGPUCheckpointCallback(
            os.path.join(weights_dir, 'azure_chest_xray_14_weights_712split_epoch_{epoch:03d}_val_loss_{val_loss:.4f}.hdf5'),
            model_single_gpu, monitor='val_loss', save_weights_only=True)

    else:
        print("using single GPU")
        model_multi_gpu = build_model()
        model_checkpoint = ModelCheckpoint(
            os.path.join(weights_dir, 'azure_chest_xray_14_weights_712split_epoch_{epoch:03d}_val_loss_{val_loss:.4f}.hdf5'),
            monitor='val_loss', save_weights_only=False)


    num_workers = num_gpu*2 #*10

    model_multi_gpu.compile(optimizer=Adam(lr=initial_lr), loss=unweighted_binary_crossentropy)

    reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-6)

    callbacks = [model_checkpoint, reduce_lr_on_plateau]

    with open(label_path, 'rb') as f:
        labels = pickle.load(f)

    with open(partition_path, 'rb') as f:
        partition = pickle.load(f)
    
    print("BATCH SIZE 1", batch_size)
    model_multi_gpu.fit_generator(generator=DataGenSequence(labels, partition['train'], current_state='train', batch_size = batch_size),
                                epochs=epochs,
                                verbose=1,
                                callbacks=callbacks,
                                workers=num_workers,
                                # max_queue_size=32,
                                # shuffle=False,
                                validation_data=DataGenSequence(labels, partition['valid'], current_state='validation', batch_size = batch_size)
                                # validation_steps=1
                                )







## Configure and Run

Set up the run configuration.

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

run_config_2 = RunConfiguration(framework = "python")
run_config_2.target = gpu_cluster_name
run_config_2.environment.docker.enabled = True

run_config_2.environment.docker.base_image = 'kateyuan/chestxraynoaml:1.0.4' #DEFAULT_CPU_IMAGE #'nvidia/cuda:9.0-cudnn7-devel' 
print('Base Docker image is:', run_config_2.environment.docker.base_image )

run_config_2.environment.python.user_managed_dependencies = True
run_config_2.environment.python.interpreter_path = '/opt/conda/bin/python'

Create Estimator object and mount datastore.

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Run

script_params = {
    '--data-folder': ds.as_mount(),
}


est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=gpu_cluster,
                entry_script='train.py',
                environment_definition=run_config_2.environment)


Run the training.

In [ ]:
run = exp.submit(config=est)
run.wait_for_completion(show_output = True)
run.get_portal_url()
print("finished")

In [ ]:
#Download models (optional)
# ds.download(target_path='./models',
#             prefix='data/chestxray/output/weights_tmpdir',
#             show_progress=True)